In [98]:
import pandas as pd
from tqdm import tqdm
import re
import json

In [2]:
top_50 = pd.read_csv("./TOP_50_CODES.csv", header=None)

In [3]:
top_50.head()

,0
0,E785
1,I10
2,Z87891
3,K219
4,F329


In [4]:
res = []
with open("../../mimic_iv/MRCONSO.RRF", mode="r") as file:
    for line in tqdm(file):
        res.append(line.split('|')[:-1])

13501908it [00:47, 281530.82it/s]


In [5]:
# https://www.ncbi.nlm.nih.gov/books/NBK9685/#ch03.sec3.3.4
cols = ["CUI", "LAT", "TS", "LUI", "STT", "SUI", "ISPREF", "AUI", "SAUI", "SCUI", "SDUI", "SAB", "TTY", "CODE", "STR", "SRL", "SUPPRESS", "CVF"]

In [6]:
res_df = pd.DataFrame(res, columns=cols)
res_df.head()

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0,N,256
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,,M0019694,D012711,MSH,ET,D012711,(131)I-MAA,0,N,256
2,C0000005,FRE,P,L6220710,PF,S7133957,Y,A13433185,,M0019694,D012711,MSHFRE,PEP,D012711,Macroagrégats d'albumine marquée à l'iode 131,3,N,
3,C0000005,FRE,S,L6215648,PF,S7133916,Y,A27488794,,M0019694,D012711,MSHFRE,ET,D012711,MAA-I 131,3,N,
4,C0000005,FRE,S,L6215656,PF,S7133956,Y,A27614225,,M0019694,D012711,MSHFRE,ET,D012711,Macroagrégats d'albumine humaine marquée à l'i...,3,N,


In [8]:
def prepare_icd(string):
    return re.sub(r'[^a-zA-Z0-9]', '', string)

In [9]:
res_df["CODE_CLEARED"] = res_df.CODE.apply(lambda x: prepare_icd(x))

In [27]:
# base_icd_cui ={}
# for icd in tqdm(top_50[0].values):
#     base_icd_cui[icd] = list(set(res_df_icd.loc[(res_df_icd.CODE_CLEARED == icd)&(res_df_icd.SAB == "ICD10CM"), "CUI"]))
#     if len(base_icd_cui[icd]) == 0:
#         base_icd_cui[icd] = list(set(res_df_icd.loc[(res_df_icd.CODE_CLEARED == icd)&(res_df_icd.SAB == "ICD10PCS"), "CUI"]))

In [63]:
# https://www.nlm.nih.gov/research/umls/sourcereleasedocs/index.html
res_df_icd = res_df[(res_df.SAB.str.contains("ICD10"))&(res_df.LAT=="ENG")]

icd_cui ={}
for icd in tqdm(top_50[0].values):
    icd_cui[icd] = list(set(res_df_icd.loc[(res_df_icd.CODE_CLEARED.str.contains(icd)), "CUI"]))
    assert len(icd_cui[icd]) > 0

100%|██████████| 50/50 [00:08<00:00,  6.18it/s]


In [85]:
icd_texts = {key: [] for key in icd_cui}
for key in tqdm(icd_cui):
    for key_i in icd_cui[key]:
        icd_texts[key].extend(res_df_icd.loc[(res_df_icd.CUI == key_i)&(~res_df_icd.TTY.isin(["PX", "HX"])), "STR"].values)

100%|██████████| 50/50 [00:12<00:00,  4.02it/s]


In [80]:
with open("./icd_names.json", 'w') as f:
    json.dump(icd_texts, f)

In [104]:
def clear_text(string):
    new_string = re.sub(r'[^a-zA-Z0-9]', ' ', string)
    return ' '.join([s.lower() for s in new_string.split() if s.lower() != "nos"])

In [105]:
icd_texts_cleared = {}
for key in icd_texts:
    icd_texts_cleared[key] = list(set(list(map(lambda x: clear_text(x), icd_texts[key]))))

In [107]:
with open("./icd_names_cleared.json", 'w') as f:
    json.dump(icd_texts_cleared, f)